## In this notebook, we will train a CNN model on the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset and use *activation maximization* to visualize the features that the trained model has learnt. 

### Step 1 Install and import all dependencies. (You can ignore the error messages in the outputs.)<br/> 

In [0]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.FATAL)
!pip install --quiet --force-reinstall git+https://github.com/raghakot/keras-vis.git
!pip install --quiet --force-reinstall scipy==1.2

In [0]:
import numpy as np
from matplotlib import pyplot as plt
from vis.utils import utils
from vis.visualization import visualize_activation, visualize_cam
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras import activations

### **Step 2** Download the data. For this exercise, we are using the MNIST dataset. This is a dataset of handwritten digits from 0 to 9.

In [0]:
# input image dimensions
img_rows, img_cols = 28, 28


# Download the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert class vectors to categorical class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### **Step 3** Visualize a sample in MNIST dataset. 

In [0]:
fig = plt.figure(figsize=(2, 2))
plt.imshow(x_train[0][:,:,0])

### **Step 4** Architect and compile your deep learning model. This is a multi-layer CNN architecture. The last layer consists of 10 filters, where each filter corresponds to a number from 0 to 9. The model classifies the input image of a handwritten digit into a number from 0 to 9, depending on which filter maximizes the output activation.
### Model below has been adapted from [Keras MNIST CNN documentation](https://keras.io/examples/mnist_cnn/).

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [0]:
# summary of model's architecture
model.summary()

### Step 5 Train your model with the loaded data.

In [0]:
batch_size = 128
epochs = 2

model.fit(x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(x_test, y_test))

### Step 6 Evaluate model accuracy with test data.


In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Step 7 Use *activation maximization* to visualize what the first filter in the last layer of our model has learnt.

In [0]:
fig = plt.figure(figsize=(2, 2))

# index of last layer
layer_idx = 7

# index of the first filter
filter_idx = 0

# modifying activation of last layer to linear 
model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)


# visualize_activation is a function in keras-vis that runs activation maximization 
# on your model and outputs an images producing maximum activation for a filter
img = visualize_activation(model, layer_idx, filter_indices=filter_idx, max_iter=1000, tv_weight=1., lp_norm_weight=0.)
plt.imshow(img[:,:,0])

### Step 8 Use activation maximization to visualize the images that maximize output of each filter in the last layer.

In [0]:
columns = 5
rows = 2
fig = plt.figure(figsize=(16, 16))

for i in range(0,10):
  img = visualize_activation(model, layer_idx, filter_indices=[i], max_iter=1000, tv_weight=1., lp_norm_weight=0.)
  fig.add_subplot(rows, columns, i+1)
  plt.imshow(img[:,:,0])
plt.show()